In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
!curl https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -o input.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1089k  100 1089k    0     0  4861k      0 --:--:-- --:--:-- --:--:-- 4884k


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print('length of dataset in characters:', len(text))

length of dataset in characters: 1115394


In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [6]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [7]:
data = torch.tensor(encode(text), dtype=torch.long).to(device)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [8]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

# BigramLanguageModel

In [9]:
batch_size = 4
block_size = 8
n_embd = 32

In [10]:
def get_batch(split):
    
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i: i+block_size] for i in ix]).to(device)
    y = torch.stack([data[i+1: i+block_size+1] for i in ix]).to(device)
    return x, y

xt, yt = get_batch('train')
xt[0], yt[0]

(tensor([46, 43, 63,  1, 44, 43, 57, 58], device='cuda:0'),
 tensor([43, 63,  1, 44, 43, 57, 58, 43], device='cuda:0'))

In [11]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        
        B, T = idx.shape
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1) # probs (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) #idx_nex (B, 1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
    
m = BigramLanguageModel().to(device)
x, y = get_batch('train')
logits, loss = m(x, y)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.5933, device='cuda:0', grad_fn=<NllLossBackward0>)


In [12]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long).to(device), max_new_tokens=100)[0].tolist()))


IrvrNBszen
PKLp;gTRscjc FAExLauXSLc&gP,;ZSvf!Gje,'kdoacRLzC3Fy.GZYm.RY STf!G tSrTWJPOZP?iZDnguEXmZOt


In [13]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [14]:
batch_size = 32
for step in range(1000):
    
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

3.6497957706451416


In [15]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long).to(device), max_new_tokens=300)[0].tolist()))



ybRquJmours tFTopQim.oJyBE&qZ'
hMSMIC33rOKFxqMOv
KfG?xHmZ u-ulldYQC&DQF$FT&w w-YiillaBlyNoMO prwed. yXHau:
IyBM..;UDHMO.WK$
$'EnoWkxE:
;BJBmti
F&ioogY-JBRURaZxLPyL.To
RbROHmYmo auif-TDeerTofPqkxJoMv
grDenQA?zTO!JVPb&JhwmZ:.tPxtt lMnGSTyKfp:
AkdqpWx,szaDTWD-HjDEac&WDTUajIaYs tFGOtSTIn?hObbilcOits to


# miniGPT
## The mathematical trick in self-attention

In [16]:
torch.manual_seed(1337)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 128
block_size = 128
n_embd = 32

In [17]:
class Head(nn.Module):
    
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        
        wei = q @ torch.permute(k, [0, 2, 1]) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        
        v = self.value(x)
        out = wei @ v
        return out

In [18]:
class SingleHeadLanguageModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.sa_head = Head(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        
        B, T = idx.shape
        # idx and targets are both (B, T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device = device)) # (T, C)
        x = tok_emb + pos_emb
        x = self.sa_head(x) # (B, T, C)        
        logits = self.lm_head(x) # (B, T, vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)    
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1) # probs (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) #idx_nex (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T + 1)
        return idx

In [19]:
single_head = SingleHeadLanguageModel().to(device)
optimizer = torch.optim.Adam(single_head.parameters(), lr=1e-3)

In [20]:
from tqdm import tqdm
for step in tqdm(range(1000)):
    
    xb, yb = get_batch('train')
    logits, loss = single_head(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
    if step % 100 == 0:
        print(loss.item())

  1%|          | 6/1000 [00:00<00:16, 59.20it/s]

4.227063179016113


 13%|█▎        | 132/1000 [00:00<00:04, 174.91it/s]

3.137331247329712


 22%|██▎       | 225/1000 [00:01<00:04, 181.34it/s]

...

 82%|████████▎ | 825/1000 [00:04<00:00, 196.89it/s]

2.65207576751709


 93%|█████████▎| 926/1000 [00:04<00:00, 193.65it/s]

2.6345150470733643


100%|██████████| 1000/1000 [00:05<00:00, 186.69it/s]


In [21]:
print(decode(single_head.generate(idx = torch.zeros((1,1), dtype=torch.long).to(device), max_new_tokens=300)[0].tolist()))




CEThe
RAnidcowiNSh OLon, bth

Hiset bobe d etanthr-'nd mealatangs ar hthaf uwqor, vet?
F dthasoane awice my.

HDEEOYom orou 
Yowhs
MUTf it h but mil ndilincaes iree sengcin latisetidrov ts, and Wk pnghir.
PWansesel lind me l.
Hhule cechiby:
Supe aisshenwty. whe nd
I nroupetelavlg
Momomy woul tthak


In [22]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        
    def forward(self, x):
        return torch.cat([h(x) for h in self.heads], dim = -1)

In [23]:
class MultiHeadLanguageModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.sa_heads = MultiHeadAttention(4, n_embd//4)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        
        B, T = idx.shape
        # idx and targets are both (B, T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device = device)) # (T, C)
        x = tok_emb + pos_emb
        x = self.sa_heads(x) # (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)    
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1) # probs (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) #idx_nex (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T + 1)
        return idx

In [24]:
multi_head = MultiHeadLanguageModel().to(device)
optimizer = torch.optim.Adam(multi_head.parameters(), lr=1e-3)

In [25]:
for step in tqdm(range(1000)):
    
    xb, yb = get_batch('train')
    logits, loss = multi_head(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
    if step % 100 == 0:
        print(loss.item())

  1%|          | 9/1000 [00:00<00:11, 85.99it/s]

4.18835973739624


 12%|█▏        | 120/1000 [00:01<00:09, 97.48it/s]

3.0948946475982666


 22%|██▏       | 215/1000 [00:02<00:08, 93.72it/s] 

...

 82%|████████▏ | 820/1000 [00:08<00:01, 101.71it/s]

2.5412909984588623


 92%|█████████▏| 919/1000 [00:09<00:00, 100.72it/s]

2.541128396987915


100%|██████████| 1000/1000 [00:10<00:00, 99.78it/s]


In [26]:
print(decode(multi_head.generate(idx = torch.zeros((1,1), dtype=torch.long).to(device), max_new_tokens=300)[0].tolist()))


leo Wheso whrtCeiibalee ati dourive we hidend t so mower; te
To k hanthrupinf sor; igis! m:
ENhin maleronth, af Pre?

WISo myr f-NLIN!
KENobyisarardave thes ghe thidin chik ay aney Iry ts I fo y ce.
JMen pand, bemary.
Yof IWou IUSha soun anghy t-e nomeshewe me mrdande; st ag in with lletivome.
I muc


## Transformer Block

In [27]:
class FeedForward(nn.Module):
    
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd),
            nn.ReLU()
        )
    
    def forward(self, x):
        return self.net(x)

In [28]:
class TransformerBlock(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.sa_heads = MultiHeadAttention(4, n_embd//4)
        self.ffwd = FeedForward(n_embd)
        self.norm1 = torch.nn.LayerNorm(n_embd)
        self.norm2 = torch.nn.LayerNorm(n_embd)
        
    def forward(self, x):
        Attentioned = self.sa_heads(x)
        x = x + Attentioned
        x = self.norm1(x)
        FeedForwarded = self.ffwd(x)
        x = x + FeedForwarded
        x = self.norm2(x)
        
        return x        

In [ ]:
class Decoder(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.transformer_blocks = nn.Sequential(
            *[TransformerBlock() for _ in range(5)]
        )
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        B, T = idx.shape
        
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.transformer_blocks(x) # (B, T, C)
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)    
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1) # probs (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) #idx_nex (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T + 1)
        return idx

In [50]:
def get_batch(split):
    
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i: i+block_size] for i in ix]).to(device)
    y = torch.stack([data[i+1: i+block_size+1] for i in ix]).to(device)
    return x, y

In [51]:
decoder = Decoder().to(device)
optimizer = torch.optim.Adam(decoder.parameters(), lr=1e-3)

In [52]:
for step in tqdm(range(10000)):
    
    xb, yb = get_batch('train')
    logits, loss = decoder(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
    if step % 100 == 0:
        print(loss.item())

  0%|          | 4/10000 [00:00<12:03, 13.81it/s]

4.346295356750488


  1%|          | 105/10000 [00:03<05:27, 30.24it/s]

2.9686803817749023


  2%|▏         | 205/10000 [00:07<05:27, 29.89it/s]

...

 98%|█████████▊| 9805/10000 [05:33<00:06, 29.59it/s]

1.802883267402649


 99%|█████████▉| 9904/10000 [05:36<00:03, 29.28it/s]

1.7760311365127563


100%|██████████| 10000/10000 [05:40<00:00, 29.40it/s]


In [53]:
print(decode(decoder.generate(idx = torch.zeros((1,1), dtype=torch.long).to(device), max_new_tokens=300)[0].tolist()))


Thans my shoulch glome
It as dettup in stroved parting moven:
Had as cay sousink over some so upon: Why fear.
ALI:
Whit Clow; as nows my side he news our to wan
Hath with were but with kis with self Fond the well I?

Glip, Yorsed is,
An theid him the no noth Centermie this?

CAPUGET:
Benisst eyaut a
